In [1]:
import numpy as np
import tensorflow as tf

from model import *
from dataset import *

C:\Users\kjosev\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# set up options

options = {}
options['margin'] = 50000
options['lrate'] = 0.05
options['dim'] = 250
options['epochs'] = 50
options['batch_size'] = 256
options['dataset'] = 'awa2'
options['num_base_classes'] = 50
options['abs'] = True

In [3]:
# fixing the seed
np.random.seed(12345)
tf.set_random_seed(12345)

In [4]:
# HYPERNYM PREDICTION DATA
hypernyms, name2index = load_hypernym_dataset(options['dataset'], 
                                              options['num_base_classes'])

options['num_all_classes'] = len(name2index)

# one hot encoding for each class with hierarchy 
hypernyms_per_class = get_hypernyms_per_class(
        hypernyms, options['num_base_classes'], options['num_all_classes'])

print('Number of base and parent classes:', len(name2index))
print('Number of hypernym pairs:', len(hypernyms))

Number of base and parent classes: 106
Number of hypernym pairs: 1332


In [5]:
# IMAGE CLASSIFICATION DATA
X, labels = load_data()
X_train, X_val, X_test, y_train, y_val, y_test = split_data(X, labels)

print('Dataset size:', X.shape)
print('Train size:', len(X_train))
print('Validation size:', len(X_val))
print('Test size:', len(X_test))

Dataset size: (37322, 4096)
Train size: 29857
Validation size: 3733
Test size: 3732


In [6]:
# Setting up our Adam optimizer 
optimizer = tf.train.AdamOptimizer(learning_rate=options['lrate'])

In [7]:
# HYPERNYM PREDICTION MODEL
pos_ch = tf.placeholder(tf.int64, shape=[None])
pos_pr = tf.placeholder(tf.int64, shape=[None])
neg_ch = tf.placeholder(tf.int64, shape=[None])
neg_pr = tf.placeholder(tf.int64, shape=[None])

hypernym_model = get_hypernym_model(pos_ch, pos_pr, neg_ch, neg_pr, options)

(h_acc, pos_acc, neg_acc), _, h_loss = hypernym_model

# hypernym prediction model only updates W_c (class) weights   
h_train_op = optimizer.minimize(h_loss)

In [8]:
# IMAGE CLASSIFICATION MODEL
im = tf.placeholder(tf.float64, shape=[None, X.shape[1]])
y = tf.placeholder(tf.int32, shape=[None])

classification_model = get_classification_model(im, y, options)

cls_acc, _, cls_loss, cls_pred = classification_model

# top 10 predictions
flat_hit_pred = get_prediction(im, 10, options) 

# get errors for all classes
cls_all_errors = get_classification_errors_all_classes(im, options) 

# image classification model updates W_i (image) and W_c (class) weights    
cls_train_op = optimizer.minimize(cls_loss) 

In [9]:
# define our tf session
sess = tf.Session()


# initialize weights
init = tf.global_variables_initializer()
sess.run(init)
sess.run(tf.local_variables_initializer())

In [10]:
# Train the hypernym prediction model

hp_steps = 0
ic_steps = 0

h_p_acc_list = []
h_n_acc_list = []
h_acc_list = []
h_loss_list = []

ic_loss_list = []
ic_train_acc_list = []
ic_val_acc_list = []

for epoch in range(1, options['epochs']):
    print("Epoch:", epoch)
    
    print("========== HYPERNYM PREDICTION ==========")
    
    h_p_acc_avg = 0
    h_n_acc_avg = 0
    h_acc_avg = 0
    h_loss_avg = 0

    for batch_idxs in get_batch_idxs(len(hypernyms), options['batch_size']):
        positive = hypernyms[batch_idxs]
        negative = generate_negative_hypernyms(len(positive), options['num_all_classes'])

        #feed to training and get results
        _, curr_loss, curr_p_acc, curr_n_acc, curr_acc = sess.run(
            [h_train_op, h_loss, pos_acc, neg_acc, h_acc], feed_dict= {
                pos_ch: positive[:,0], # children
                pos_pr: positive[:,1], # parents
                neg_ch: negative[:,0],
                neg_pr: negative[:,1]
            })
    
        # update average loss and accuracy
        h_p_acc_avg += curr_p_acc * len(batch_idxs) / float(len(hypernyms))
        h_n_acc_avg += curr_n_acc * len(batch_idxs) / float(len(hypernyms))
        h_acc_avg += curr_acc * len(batch_idxs) / float(len(hypernyms))
        h_loss_avg += curr_loss * len(batch_idxs) / float(len(hypernyms))

        if hp_steps % 10 == 0:
            print("Steps: %05d Currernt loss %f " % (hp_steps, curr_loss))

        hp_steps += 1

    print("Train Positive Accuracy: ", h_p_acc_avg )
    print("Train Negative Accuracy: ", h_n_acc_avg)
    print("Train Accuracy: ", h_acc_avg)
    
    
    h_p_acc_list.append(h_p_acc_avg)
    h_n_acc_list.append(h_n_acc_avg)
    h_acc_list.append(h_acc_avg)
    h_loss_list.append(h_loss_avg)
    
    print("========== IMAGE CLASSIFICATION ==========")
    
    # variables to keep track of our average loss and accuracy
    ic_loss_avg = 0
    train_acc_avg = 0

    # run batched training
    for batch_idxs in get_batch_idxs(len(X_train), options['batch_size']):
        
        _, accur, curr_cls_loss = sess.run(
            [cls_train_op, cls_acc, cls_loss], feed_dict= {
                im: X_train[batch_idxs], 
                y: y_train[batch_idxs]
            })
        
        ic_steps += 1
        
        # update average loss and accuracy
        ic_loss_avg += curr_cls_loss * len(batch_idxs) / float(len(X_train))
        train_acc_avg += accur * len(batch_idxs) / float(len(X_train))

        if ic_steps % 10 == 0:
            print("Steps: %05d Accuracy: %f" % (
                ic_steps, accur))

    print()
    print("Train Accuracy: %f Loss: %f\n" % (
            train_acc_avg, 
            ic_loss_avg
        )
    )
    
    ic_loss_list.append(ic_loss_avg)
    ic_train_acc_list.append(train_acc_avg)
    
    # variables to keep track of our average val loss and accuracy
    val_acc_avg = 0
    
    # get validation accuracy in batches
    for batch_idxs in get_batch_idxs(len(X_val), options['batch_size']):
        [curr_val_accur] = sess.run(
            [cls_acc], feed_dict={ 
                im: X_val[batch_idxs], 
                y: y_val[batch_idxs]
            }
        )

        val_acc_avg += curr_val_accur * len(batch_idxs) / float(len(X_val))

    print("\nValidation Accuracy: %f \n" % (val_acc_avg))
    
    ic_val_acc_list.append(val_acc_avg)

Epoch: 1
========== HYPERNYM PREDICTION ==========
Steps: 00000 Currernt loss 12799999.041901 
Train Positive Accuracy:  1.0
Train Negative Accuracy:  0.0
Train Accuracy:  0.5
========== IMAGE CLASSIFICATION ==========
Steps: 00010 Accuracy: 0.039062
Steps: 00020 Accuracy: 0.156250
Steps: 00030 Accuracy: 0.132812
Steps: 00040 Accuracy: 0.144531
Steps: 00050 Accuracy: 0.164062
Steps: 00060 Accuracy: 0.148438
Steps: 00070 Accuracy: 0.187500
Steps: 00080 Accuracy: 0.238281
Steps: 00090 Accuracy: 0.214844
Steps: 00100 Accuracy: 0.199219
Steps: 00110 Accuracy: 0.250000

Train Accuracy: 0.160733 Loss: 575956891.703763


Validation Accuracy: 0.270828 

Epoch: 2
========== HYPERNYM PREDICTION ==========
Steps: 00010 Currernt loss 12025010.065472 
Train Positive Accuracy:  1.0
Train Negative Accuracy:  0.0
Train Accuracy:  0.5
========== IMAGE CLASSIFICATION ==========
Steps: 00120 Accuracy: 0.273438
Steps: 00130 Accuracy: 0.289062
Steps: 00140 Accuracy: 0.308594
Steps: 00150 Accuracy: 0.367188

Steps: 01410 Accuracy: 0.871094
Steps: 01420 Accuracy: 0.914062
Steps: 01430 Accuracy: 0.875000
Steps: 01440 Accuracy: 0.855469
Steps: 01450 Accuracy: 0.886719
Steps: 01460 Accuracy: 0.867188
Steps: 01470 Accuracy: 0.851562
Steps: 01480 Accuracy: 0.906250
Steps: 01490 Accuracy: 0.890625
Steps: 01500 Accuracy: 0.898438
Steps: 01510 Accuracy: 0.875000
Steps: 01520 Accuracy: 0.882812

Train Accuracy: 0.886727 Loss: 10950646.529556


Validation Accuracy: 0.871953 

Epoch: 14
========== HYPERNYM PREDICTION ==========
Steps: 00080 Currernt loss 4930178.695811 
Train Positive Accuracy:  0.9992492492492493
Train Negative Accuracy:  0.45345345345345345
Train Accuracy:  0.7263513513513513
========== IMAGE CLASSIFICATION ==========
Steps: 01530 Accuracy: 0.855469
Steps: 01540 Accuracy: 0.914062
Steps: 01550 Accuracy: 0.890625
Steps: 01560 Accuracy: 0.898438
Steps: 01570 Accuracy: 0.890625
Steps: 01580 Accuracy: 0.902344
Steps: 01590 Accuracy: 0.894531
Steps: 01600 Accuracy: 0.875000
Steps: 01610 

Train Positive Accuracy:  0.9962462462462462
Train Negative Accuracy:  0.6298798798798799
Train Accuracy:  0.8130630630630631
========== IMAGE CLASSIFICATION ==========
Steps: 02810 Accuracy: 0.949219
Steps: 02820 Accuracy: 0.917969
Steps: 02830 Accuracy: 0.953125
Steps: 02840 Accuracy: 0.941406
Steps: 02850 Accuracy: 0.937500
Steps: 02860 Accuracy: 0.902344
Steps: 02870 Accuracy: 0.917969
Steps: 02880 Accuracy: 0.925781
Steps: 02890 Accuracy: 0.945312
Steps: 02900 Accuracy: 0.941406
Steps: 02910 Accuracy: 0.937500
Steps: 02920 Accuracy: 0.953125

Train Accuracy: 0.941756 Loss: 3516729.463838


Validation Accuracy: 0.894455 

Epoch: 26
========== HYPERNYM PREDICTION ==========
Steps: 00150 Currernt loss 3124743.568317 
Train Positive Accuracy:  0.9962462462462461
Train Negative Accuracy:  0.6268768768768768
Train Accuracy:  0.8115615615615617
========== IMAGE CLASSIFICATION ==========
Steps: 02930 Accuracy: 0.945312
Steps: 02940 Accuracy: 0.941406
Steps: 02950 Accuracy: 0.937500
Steps:

Steps: 04190 Accuracy: 0.941406
Steps: 04200 Accuracy: 0.980469
Steps: 04210 Accuracy: 0.980469

Train Accuracy: 0.966574 Loss: 1672058.529666


Validation Accuracy: 0.900080 

Epoch: 37
========== HYPERNYM PREDICTION ==========
Steps: 00220 Currernt loss 2291488.633105 
Train Positive Accuracy:  0.9977477477477477
Train Negative Accuracy:  0.7507507507507508
Train Accuracy:  0.8742492492492492
========== IMAGE CLASSIFICATION ==========
Steps: 04220 Accuracy: 0.964844
Steps: 04230 Accuracy: 0.972656
Steps: 04240 Accuracy: 0.972656
Steps: 04250 Accuracy: 0.976562
Steps: 04260 Accuracy: 0.945312
Steps: 04270 Accuracy: 0.964844
Steps: 04280 Accuracy: 0.980469
Steps: 04290 Accuracy: 0.976562
Steps: 04300 Accuracy: 0.953125
Steps: 04310 Accuracy: 0.964844
Steps: 04320 Accuracy: 0.960938

Train Accuracy: 0.967076 Loss: 1601195.451527


Validation Accuracy: 0.896866 

Epoch: 38
========== HYPERNYM PREDICTION ==========
Train Positive Accuracy:  0.9977477477477478
Train Negative Accuracy:  0.7

Steps: 05530 Accuracy: 0.976562
Steps: 05540 Accuracy: 0.972656
Steps: 05550 Accuracy: 0.980469
Steps: 05560 Accuracy: 0.980469
Steps: 05570 Accuracy: 0.972656
Steps: 05580 Accuracy: 0.980469
Steps: 05590 Accuracy: 0.988281
Steps: 05600 Accuracy: 0.996094
Steps: 05610 Accuracy: 0.968750

Train Accuracy: 0.976756 Loss: 1076709.950350


Validation Accuracy: 0.900884 

Epoch: 49
========== HYPERNYM PREDICTION ==========
Steps: 00290 Currernt loss 2252705.919186 
Train Positive Accuracy:  0.9992492492492492
Train Negative Accuracy:  0.792042042042042
Train Accuracy:  0.8956456456456456
========== IMAGE CLASSIFICATION ==========
Steps: 05620 Accuracy: 0.984375
Steps: 05630 Accuracy: 0.960938
Steps: 05640 Accuracy: 0.984375
Steps: 05650 Accuracy: 0.972656
Steps: 05660 Accuracy: 0.988281
Steps: 05670 Accuracy: 0.980469
Steps: 05680 Accuracy: 0.976562
Steps: 05690 Accuracy: 0.968750
Steps: 05700 Accuracy: 0.992188
Steps: 05710 Accuracy: 0.976562
Steps: 05720 Accuracy: 0.972656
Steps: 05730 Acc

In [11]:
#Calculate hierarchical recall and precision

index2name = {v:k for k,v in name2index.items()}

ground_truth = []
predicted = []
idxs_order = []

val_errors = []
for batch_idxs in get_batch_idxs(len(X_val), options['batch_size']):
    [val_pred, curr_val_errors] = sess.run(
        [flat_hit_pred, cls_all_errors], feed_dict={ 
            im: X_val[batch_idxs], 
            y: y_val[batch_idxs]
        }
    )

    ground_truth.extend(y_val[batch_idxs])
    predicted.extend(val_pred)

    val_errors.extend(curr_val_errors)

y_val_hier = [hypernyms_per_class[int(l)] for l in ground_truth]

prec = []
recall = []
for idx in range(len(X_val)):
    pred_hier = val_errors[idx] <= min(val_errors[idx][:options['num_base_classes']])
    curr_precision = sum(y_val_hier[idx] * pred_hier) / sum(pred_hier)
    curr_recall = sum(y_val_hier[idx] * pred_hier) / sum(y_val_hier[idx])

    prec.append(curr_precision)
    recall.append(curr_recall)

print("Hierarchial results:\tPrecision: %.2f\tRecall: %.2f" % (np.mean(prec),np.mean(recall)))

Hierarchial results:	Precision: 0.79	Recall: 0.93


In [12]:
# calculate flat hit accuracy
flat_hit_at_1 = 0
flat_hit_at_3 = 0
flat_hit_at_5 = 0
flat_hit_at_10 = 0

for i in range(len(X_val)):
    try:
        found_idx = list(predicted[i]).index(ground_truth[i])
    except:
        found_idx = 9999999

    if found_idx < 1:
        flat_hit_at_1 += 1.0 / len(X_val)
    if found_idx < 3:
        flat_hit_at_3 += 1.0 / len(X_val)
    if found_idx < 5:
        flat_hit_at_5 += 1.0 / len(X_val)
    if found_idx < 10:
        flat_hit_at_10 += 1.0 / len(X_val)


print("Flat hit accuracy:\t@ 1: %.4f\t@ 3: %.4f\t@ 5: %.4f\t@ 10: %.4f" % (
    flat_hit_at_1,
    flat_hit_at_3,
    flat_hit_at_5,
    flat_hit_at_10
))

Flat hit accuracy:	@ 1: 0.9025	@ 3: 0.9700	@ 5: 0.9834	@ 10: 0.9938
